In [21]:
rounddown(a) = round(a, RoundDown)

function quadrant_travelcost(m) 
    # cost: cheapest cost from top left corner to every other square restricted to only South/East moves. (quadrant 4)
    # and
    # dir: correct FIRST direction to go in order to take that cheapest path
    
    cost = m*0.1
    
    Y=size(m,1)
    X=size(m,2)
    dir=ones(Int,(Y,X))
    sumcost=zeros(Int,(Y,X))
    for x=1:X,y=1:Y
        if x>1
            sumcost[y,x] = sumcost[y,x-1] + rounddown(cost[y,x-1])
            dir[y,x]  = y==1 ? 1 : dir[y,x-1]
        end
        if y>1
            t_sumcost = sumcost[y-1,x] + rounddown(cost[y-1,x])
            if t_sumcost<sumcost[y,x] || x==1
                sumcost[y,x]=t_sumcost
                dir[y,x] = x==1 ? 0 : dir[y-1,x]
            end
        end
    end
    dir[1,1]=4
    
    return sumcost, dir
end

q1(m) = reverse(circshift(m, (-1,0)), dims=1)
q2(m) = q1(q3(m))
q3(m) = reverse(circshift(m, (0,-1)), dims=2)
q4(m) = m

iq1(m,d) = q1(m), replace(d, 0 => 2)
iq2(m,d) = q2(m), replace(d, 0 => 2, 1 => 3)
iq3(m,d) = q3(m), replace(d, 1 => 3)
iq4(m,d) = q4(m), d

shiftorigin(m, origin) = circshift(m, (1-origin[1], 1-origin[2]))
ishiftorigin(m, origin) = circshift(m, (origin[1]-1, origin[2]-1))


function travelcost(m, origin)
    #Cheapest cost from some point to All Other points,
    #and what direction to go on First step to take that cheapest path.
    #south/east/north/west (in that order 0,1,2,3)
    m = shiftorigin(m, origin)
    expand(f) = x -> f(x...)
    c1,d1 = m |> q1 |> quadrant_travelcost |> expand(iq1)
    c2,d2 = m |> q2 |> quadrant_travelcost |> expand(iq2)
    c3,d3 = m |> q3 |> quadrant_travelcost |> expand(iq3)
    c4,d4 = m |> q4 |> quadrant_travelcost |> expand(iq4)
    
    D = cat(d1,d2,d3,d4, dims=3)
    C = cat(c1,c2,c3,c4, dims=3)
    v, i = findmin(C, dims=3)
    
    cost = ishiftorigin(C[i][:,:,1], origin)
    first_direction = ishiftorigin(D[i][:,:,1], origin) #pick the index with least cost. distance not taken into account..
    
    
    return cost, first_direction
end

manhattandist(a,b) = sum(abs.(b-a))

function manhattandistmatrix(m,origin)
    #with wrap around taken into account
    Y=size(m,1)
    X=size(m,2)
    mhdist_q4=zeros(Int,(Y,X))
    mhdist_q1=zeros(Int,(Y,X))
    mhdist_q2=zeros(Int,(Y,X))
    mhdist_q3=zeros(Int,(Y,X))
    o=[origin[1],origin[2]]
    for x=1:X, y=1:Y
        mhdist_q4[y,x] = manhattandist([1,1],[y,x])
        mhdist_q3[y,x] = manhattandist([1,X],[y,x])
        mhdist_q2[y,x] = manhattandist([Y,X],[y,x])
        mhdist_q1[y,x] = manhattandist([Y,1],[y,x])
    end
    mhdist_q4 = ishiftorigin(mhdist_q4, origin)
    mhdist_q3 = ishiftorigin(shiftorigin(mhdist_q3, [1,X]), origin)
    mhdist_q2 = ishiftorigin(shiftorigin(mhdist_q2, [Y,X]), origin)
    mhdist_q1 = ishiftorigin(shiftorigin(mhdist_q1, [Y,1]), origin)
    
    return min.(mhdist_q4,mhdist_q3,mhdist_q2,mhdist_q1)
end

function halite_per_turn(m, p_ship, p_shipyard, ship_halite)
    #mined halite amount (at some point)
    #minus
    #cost (from ship, to some point, and then to shipyard)
    #divided by
    #nr turns to travel there and then to shipyard
    #equals
    #net halite gained per turn
    
    mining = m .* 0.25
    cost1, direction1 = travelcost(m, p_ship)
    cost2, direction2 = travelcost(m, p_shipyard)
    cost2 = cost2 + (m*0.75)*0.1
    
    cost = cost1 + cost2
    net_gain = mining - cost
    net_gain[p_shipyard[1],p_shipyard[2]] = ship_halite - cost1[p_shipyard[1],p_shipyard[2]]
    
    mhd1 = manhattandistmatrix(m, p_ship)
    mhd2 = manhattandistmatrix(m, p_shipyard)
    mhd = mhd1 .+ mhd2
    #hpt = net_gain./mhd
    
    hpt = net_gain ./ (mhd.+1) #plus 1 since we mined one turn
    hpt[p_shipyard[1],p_shipyard[2]] = net_gain[p_shipyard[1],p_shipyard[2]] ./ mhd[p_shipyard[1],p_shipyard[2]]
    
    if p_ship == p_shipyard
        hpt[p_shipyard[1],p_shipyard[2]] = 0
    end
    
    return hpt, cost1, direction1
end

within_reach(hpt, cost1, ship_halite) = hpt.*(cost1 .<= ship_halite)

function select_direction(m, p_ship, p_shipyard, ship_halite)
    hpt, cost1, direction1 = halite_per_turn(m, p_ship, p_shipyard, ship_halite)
    hpt_within_reach = within_reach(hpt, cost1, ship_halite)
    dir = direction1[findmax(hpt_within_reach)[2]]
    return dir
end


select_direction (generic function with 1 method)

In [22]:
p_ship = [2,1]
p_shipyard = [2,2]
#ship_halite = 39-39*0.75^1
ship_halite = 0

m=[90 0 0 0;
    0 0 0 0;
    0 0 0 0;
    0 0 0 0]

m[p_shipyard[1],p_shipyard[2]]=0 #(make sure shipyard has 0 halite when constructing matrix manually)
m

4×4 Array{Int64,2}:
 90  0  0  0
  0  0  0  0
  0  0  0  0
  0  0  0  0

In [23]:
select_direction(m, p_ship, p_shipyard, ship_halite)

2

In [24]:
p_ship = [1,1]
select_direction(m, p_ship, p_shipyard, ship_halite)

4

In [30]:
#m[1,1] = floor(m[1,1]*0.75)
m

4×4 Array{Int64,2}:
 67  0  0  0
  0  0  0  0
  0  0  0  0
  0  0  0  0

In [31]:
ship_halite = 23

23

In [32]:
select_direction(m, p_ship, p_shipyard, ship_halite)

2

In [33]:
#for inspection:
hpt, cost1, direction1 = halite_per_turn(m, p_ship, p_shipyard, ship_halite)
hpt

4×4 Array{Float64,2}:
  3.90833  -2.0  -1.2       -1.2     
 -2.0       8.5  -1.2       -1.2     
 -1.2      -1.2  -0.857143  -0.857143
 -1.2      -1.2  -0.857143  -0.857143

In [34]:
hpt_within_reach = within_reach(hpt, cost1, ship_halite)

4×4 Array{Float64,2}:
  3.90833  -2.0  -1.2       -1.2     
 -2.0       8.5  -1.2       -1.2     
 -1.2      -1.2  -0.857143  -0.857143
 -1.2      -1.2  -0.857143  -0.857143

In [35]:
direction1

4×4 Array{Int64,2}:
 4  1  1  1
 2  2  2  2
 2  2  2  2
 2  2  2  2

In [36]:
#cost is same to all, it should be. but when selecting direction need to take into account distance aswel
#this needs to be done inside travelcost() function which returns direction
cost1

4×4 Array{Int64,2}:
 0  6  6  6
 6  6  6  6
 6  6  6  6
 6  6  6  6

In [15]:
c, d = travelcost(m, p_ship)
d

4×4 Array{Int64,2}:
 4  1  1  1
 2  2  2  2
 2  2  2  2
 2  2  2  2

In [245]:
p_ship = [1,1]
p_shipyard = [1,1]
#ship_halite = 39-39*0.75^1
ship_halite = 0

m=[9 20 30;
    39*0.75^0 50*0.75^0 60*0.75^0;
    50*0.75^0 59 10;
    80*0.75^0 30 70*0.75^0]
m[p_shipyard[1],p_shipyard[2]]=0 #(make sure shipyard has 0 halite when constructing matrix manually)

m

4×3 Array{Float64,2}:
  0.0  20.0  30.0
 39.0  50.0  60.0
 50.0  59.0  10.0
 80.0  30.0  70.0

In [246]:
select_direction(m, p_ship, p_shipyard, ship_halite)

2

In [247]:
#for inspection:
hpt, cost1, direction1 = halite_per_turn(m, p_ship, p_shipyard, ship_halite)
hpt

4×3 Array{Float64,2}:
 0.0      1.16667     1.75   
 2.275    0.95        0.9    
 0.55     0.0464286  -2.03571
 4.66667  0.25        1.25   

In [244]:
cost1

4×3 Array{Int64,2}:
 0  0  0
 0  2  2
 2  5  7
 0  2  3

In [167]:
hpt_within_reach = within_reach(hpt, cost1, ship_halite)

4×3 Array{Float64,2}:
 NaN        1.16667   1.75
   2.275    0.0       0.0 
   0.0      0.0      -0.0 
   4.66667  0.0       0.0 

In [135]:
ship_halite

0

In [239]:
[1,2]==[1,3]

false

In [ ]:
M2 = [90 0 0 0
      0  0 0 0
      0  0 0 0
      0  0 0 0]